btrack can be installed with pip:

```sh
pip install btrack
```

In [1]:
import btrack
from btrack import datasets
# import napari

In [2]:
CONFIG_FILE = datasets.cell_config()
# CONFIG_FILE = datasets.particle_config()

In [3]:
objects = datasets.example_track_objects()

In [4]:
objects[0]

,ID,x,y,z,t,dummy,states,label
0,0,517.559509,9.081633,0.0,0,False,0,0


In [5]:
objects[0].to_dict()

{'ID': 0,
 'x': 517.5595092773438,
 'y': 9.081632614135742,
 'z': 0.0,
 't': 0,
 'dummy': False,
 'states': 0,
 'label': 0}

In [6]:
cfg = btrack.config.load_config(CONFIG_FILE)

[INFO][2023/03/08 02:17:22 PM] Loading configuration file: /Users/joran.deschamps/Library/Caches/btrack-examples/examples/cell_config.json


In [7]:
cfg.motion_model.prob_not_assign = 0.1
cfg.motion_model.max_lost = 1
cfg.hypothesis_model.time_thresh = 1
cfg.hypothesis_model.theta_time = 1
cfg.hypothesis_model.dist_thresh = 100

In [8]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure(cfg)
    tracker.max_search_radius = 50

    # append the objects to be tracked
    tracker.append(objects)

    # set the tracking volume
    tracker.volume=((0, 1600), (0, 1200))

    # track them (in interactive mode)
    tracker.track(step_size=100)

    # generate hypotheses and run the global optimizer
    # tracker.optimize()

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari()
    
    # store the tracks
    tracks = tracker.tracks

    # edges = tracker.graph_edges()
    # edge_one = [edge for edge in edges if edge.source == 50]
    # print([e.to_dict() for e in edge_one])
    # print(sum(e.score for e in edge_one))

    # store the configuration
    cfg = tracker.configuration

[INFO][2023/03/08 02:17:22 PM] Loaded btrack: /Users/joran.deschamps/miniconda3/envs/btrack/lib/python3.8/site-packages/btrack/libs/libtracker.dylib
[INFO][2023/03/08 02:17:22 PM] btrack (v0.5.0) library imported
[INFO][2023/03/08 02:17:23 PM] Starting BayesianTracker session
[INFO][2023/03/08 02:17:23 PM] Objects are of type: <class 'list'>
[INFO][2023/03/08 02:17:23 PM] Starting tracking... 
[INFO][2023/03/08 02:17:23 PM] Update using: ['MOTION']
[INFO][2023/03/08 02:17:23 PM] Tracking objects in frames 0 to 99 (of 500)...
[INFO][2023/03/08 02:17:23 PM]  - Timing (Bayesian updates: 3.49ms, Linking: 0.28ms)
[INFO][2023/03/08 02:17:23 PM]  - Probabilities (Link: 0.99986, Lost: 0.74200)
[INFO][2023/03/08 02:17:23 PM]  - Stats (Active: 80, Lost: 599, Conflicts resolved: 2)
[INFO][2023/03/08 02:17:23 PM] Tracking objects in frames 100 to 199 (of 500)...
[INFO][2023/03/08 02:17:23 PM]  - Timing (Bayesian updates: 3.63ms, Linking: 0.30ms)
[INFO][2023/03/08 02:17:23 PM]  - Probabilities (Lin

In [9]:
# viewer = napari.Viewer()
# # viewer.add_image(images)
# # viewer.add_points(detections.loc[:, ["t", "x", "y"]])
# viewer.add_tracks(data, properties=properties, graph=graph)

In [10]:
print(len(tracks))
print(type(tracks))

698
<class 'list'>


In [11]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy
0,1,0,517.559509,9.081633,0.0,1,1,0,0,False
1,1,1,514.647217,5.522222,0.0,1,1,2,0,False


In [12]:
tracks[0].to_dict()

OrderedDict([('ID', 1),
             ('t', [0, 1]),
             ('x', [517.5595092773438, 514.647216796875]),
             ('y', [9.081632614135742, 5.52222204208374]),
             ('z', [0.0, 0.0]),
             ('parent', 1),
             ('root', 1),
             ('state', [0, 2]),
             ('generation', 0),
             ('dummy', [False, False])])

# Data massaging

We want the following array/list:

nodes:
| node_id | t | z | x | y | ... |
| -------- | - | - | - | - | --- |

edges:
| edge_id | source | end |
| ------- | ------ | --- |

tracks:
| track_id | node_id start | node_id end | 
| -------- | ------------- | ----------- | 

division?

In [13]:
import pandas as pd

print(objects[0])

{'ID': 0, 'x': 517.5595092773438, 'y': 9.081632614135742, 'z': 0.0, 't': 0, 'dummy': False, 'states': 0, 'label': 0}


### nodes

In [14]:
nodes = []
for o in objects:
    node = [
        o.ID,
        o.t,
        o.z,
        o.x,
        o.y,
    ]
    nodes.append(node)

In [15]:
nodes_df = pd.DataFrame.from_dict(nodes)
print(nodes_df)

           0    1    2            3            4
0          0    0  0.0   517.559509     9.081633
1          1    0  0.0   657.603760     4.430189
2          2    0  0.0   718.007568     8.603536
3          3    0  0.0   688.747314    21.547287
4          4    0  0.0   234.807022    34.463657
...      ...  ...  ...          ...          ...
52576  52576  499  0.0  1150.620850  1172.218384
52577  52577  499  0.0  1490.937012  1177.104858
52578  52578  499  0.0  1051.531250  1188.209839
52579  52579  499  0.0  1002.518372  1189.372925
52580  52580  499  0.0  1529.802979  1191.474487

[52581 rows x 5 columns]


### edges and tracks

In [16]:
print(len(tracks[0]))
print(tracks[0].x[1])
print(tracks[1].refs)
tracks[1]

2
514.647216796875
[1, 70, 139, 207]


,ID,t,x,y,z,parent,root,state,generation,dummy
0,2,0,657.603760,4.430189,0.0,2,2,2,0,False
1,2,1,657.799988,4.155102,0.0,2,2,2,0,False
2,2,2,657.011658,4.298450,0.0,2,2,2,0,False
3,2,3,656.730286,4.713816,0.0,2,2,2,0,False


In [17]:
edges = []
tracklets = []

edge_id = 0
for t in tracks:
    # add track
    track = [
        t.ID,
        t.refs[0],
        t.refs[-1],
    ]
    tracklets.append(track)
    
    # add edges
    for i in range(len(t)-1):
        edge = [
            edge_id,
            t.refs[i],
            t.refs[i+1],
        ]
        
        edges.append(edge)
        edge_id += 1

In [18]:
edges_df = pd.DataFrame.from_dict(edges)
print(edges_df)

           0      1      2
0          0      0     69
1          1      1     70
2          2     70    139
3          3    139    207
4          4      2     71
...      ...    ...    ...
51878  51878  52208  52388
51879  51879  52388  52570
51880  51880  52307  52487
51881  51881  52327  52509
51882  51882  52329  52511

[51883 rows x 3 columns]


In [19]:
tracklets_df = pd.DataFrame.from_dict(tracklets)
print(tracklets_df)

       0      1      2
0      1      0     69
1      2      1    207
2      3      2    818
3      4      3  31492
4      5      4   5284
..   ...    ...    ...
693  694  52307  52487
694  695  52327  52509
695  696  52329  52511
696  697  52399  52399
697  698  52493  52493

[698 rows x 3 columns]


# Let's try exporting to zarr

In [20]:
import numpy as np
from pathlib import Path
import shutil

# verify existence|
zarr_file = Path('data/example.n5')
if zarr_file.exists():
        shutil.rmtree(zarr_file)

In [21]:
import zarr
store = zarr.N5Store(zarr_file)
root = zarr.group(store=store)

# nodes
nodes_group = root.create_group('nodes')
edges_group = root.create_group('edges')
tracks_group = root.create_group('tracks')

In [22]:
# nodes
nid = np.array(nodes_df.loc[:, 0])
nt = np.array(nodes_df.loc[:, 1])
nz = np.array(nodes_df.loc[:, 2])
nx = np.array(nodes_df.loc[:, 3])
ny = np.array(nodes_df.loc[:, 4])

nid_chunk = nodes_group.array(name='nid', data=nid, dtype='i4')
nt_chunk = nodes_group.array(name='t', data=nt, dtype='i4')
nz_chunk = nodes_group.array(name='z', data=nz, dtype='f4')
nx_chunk = nodes_group.array(name='x', data=nx, dtype='f4')
ny_chunk = nodes_group.array(name='y', data=ny, dtype='f4')

In [23]:
# edges
eid = np.array(edges_df.loc[:, 0])
source = np.array(edges_df.loc[:, 1])
target = np.array(edges_df.loc[:, 2])

eid_chunk = edges_group.array(name='eid', data=eid, dtype='i4')
source_chunk = edges_group.array(name='source', data=source, dtype='i4')
target_chunk = edges_group.array(name='target', data=target, dtype='i4')

In [24]:
# tracklets
tid = np.array(tracklets_df.loc[:, 0])
begin = np.array(tracklets_df.loc[:, 1])
end = np.array(tracklets_df.loc[:, 2])

tid_chunk = tracks_group.array(name='tid', data=tid, dtype='i4')
begin_chunk = tracks_group.array(name='begin', data=begin, dtype='i4')
end_chunk = tracks_group.array(name='end', data=end, dtype='i4')

In [25]:
print(root.tree())

/
 ├── edges
 │   ├── eid (51883,) int32
 │   ├── source (51883,) int32
 │   └── target (51883,) int32
 ├── nodes
 │   ├── nid (52581,) int32
 │   ├── t (52581,) int32
 │   ├── x (52581,) float32
 │   ├── y (52581,) float32
 │   └── z (52581,) float32
 └── tracks
     ├── begin (698,) int32
     ├── end (698,) int32
     └── tid (698,) int32


# Read zarr

In [26]:
myzarr = zarr.open(zarr_file)
print(myzarr.tree())

/
 ├── edges
 │   ├── eid (51883,) int32
 │   ├── source (51883,) int32
 │   └── target (51883,) int32
 ├── nodes
 │   ├── nid (52581,) int32
 │   ├── t (52581,) int32
 │   ├── x (52581,) float32
 │   ├── y (52581,) float32
 │   └── z (52581,) float32
 └── tracks
     ├── begin (698,) int32
     ├── end (698,) int32
     └── tid (698,) int32


In [27]:
n = 100
objects[n]

,ID,x,y,z,t,dummy,states,label
0,100,405.485748,664.952942,0.0,1,False,0,0


In [28]:
obj_id = myzarr['nodes/nid'][n]
obj_t = myzarr['nodes/t'][n]
obj_x = myzarr['nodes/x'][n]


print(f'Object 10: {obj_id}, {obj_t}, {obj_x}')

Object 10: 100, 1, 405.4857482910156


In [29]:
print(tracks[n].refs)
tracks[n]

[1751, 1822, 1892, 1962, 2035, 2108, 2181, 2253, 2327, 2400, 2473, 2545, 2618, 2691, 2764, 2836, 2908, 2980, 3052, 3124, 3196, 3268, 3340, 3414, 3487, 3560, 3634, 3708, 3780, 3854, 3927, 3999, 4073, 4146, 4219, 4292]


,ID,t,x,y,z,parent,root,state,generation,dummy
0,101,25,1002.694519,889.183289,0.0,101,101,0,0,False
1,101,26,1004.174683,889.793213,0.0,101,101,0,0,False
2,101,27,1003.641479,883.739258,0.0,101,101,0,0,False
3,101,28,1006.424316,873.554565,0.0,101,101,0,0,False
4,101,29,1005.196472,866.737366,0.0,101,101,0,0,False
5,101,30,1003.860535,860.402771,0.0,101,101,0,0,False
6,101,31,1004.886169,853.296021,0.0,101,101,0,0,False
7,101,32,1003.967651,851.726257,0.0,101,101,0,0,False
8,101,33,1003.536865,846.135071,0.0,101,101,0,0,False
9,101,34,1004.329041,843.351501,0.0,101,101,0,0,False


In [30]:
tracks_id = myzarr['tracks/tid'][n]
tracks_begin = myzarr['tracks/begin'][n]
tracks_end = myzarr['tracks/end'][n]


print(f'Object 10: {tracks_id}, {tracks_begin}, {tracks_end}')

Object 10: 101, 1751, 4292
